# Imports

In [101]:
import numpy as np
import pandas as pd

In [102]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

# Read Datasets

In [103]:
historical_results = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/historical-results.csv')
historical_results.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [104]:
historical_results = historical_results.replace('United States','USA', regex=True) # changing for pattern

In [105]:
historical_win_loose_draw_ratios = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/historical_win-loose-draw_ratios.csv')
historical_win_loose_draw_ratios.head(5)

,country1,country2,games,wins,looses,draws
0,Argentina,Australia,7,0.714286,0.142857,0.142857
1,Australia,Argentina,7,0.142857,0.714286,0.142857
2,Argentina,Belgium,4,0.750000,0.250000,0.000000
3,Belgium,Argentina,4,0.250000,0.750000,0.000000
4,Argentina,Brazil,108,0.361111,0.398148,0.240741


In [106]:
ranking = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/ranking.csv')
ranking.head(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [107]:
shootouts = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/shootouts.csv')
shootouts.head(5)

,date,home_team,away_team,winner
0,1967-08-22,India,Taiwan,Taiwan
1,1971-11-14,South Korea,Vietnam Republic,South Korea
2,1972-05-17,Thailand,South Korea,South Korea
3,1972-05-19,Thailand,Cambodia,Thailand
4,1973-04-21,Senegal,Ghana,Ghana


# Historical Results

In [108]:
historical_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [109]:
historical_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


Top 10 tournaments on database:

In [110]:
historical_results.groupby('tournament').count().sort_values('date', ascending=False)['date'].head(10)

tournament
Friendly                                17425
FIFA World Cup qualification             7774
UEFA Euro qualification                  2593
African Cup of Nations qualification     1932
FIFA World Cup                            900
Copa América                              841
AFC Asian Cup qualification               764
African Cup of Nations                    742
CECAFA Cup                                620
CFU Caribbean Cup qualification           606
Name: date, dtype: int64

In [111]:
historical_results_world_cup = historical_results.query('tournament == "FIFA World Cup" \
                                                        | home_team == "Qatar" \
                                                        | away_team == "Qatar" \
                                                        | home_team == "Wales" \
                                                        | away_team == "Wales"').sort_values('date').reset_index(drop=True)
historical_results_world_cup.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False
1,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False
2,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False
3,1879-01-18,England,Wales,2.0,1.0,Friendly,London,England,False
4,1879-04-07,Wales,Scotland,0.0,3.0,Friendly,Wrexham,Wales,False
5,1880-03-15,Wales,England,2.0,3.0,Friendly,Wrexham,Wales,False
6,1880-03-27,Scotland,Wales,5.0,1.0,Friendly,Glasgow,Scotland,False
7,1881-02-26,England,Wales,0.0,1.0,Friendly,Blackburn,England,False
8,1881-03-14,Wales,Scotland,1.0,5.0,Friendly,Wrexham,Wales,False
9,1882-02-25,Wales,Northern Ireland,7.0,1.0,Friendly,Wrexham,Wales,False


Identifying finals

In [112]:
historical_results_world_cup['is_final'] = ''

for i in range(len(historical_results_world_cup['date'])):
  if i < 899 and historical_results_world_cup[i:i+1]['date'].astype(str).str[:4][i] != historical_results_world_cup[i+1:i+2]['date'].astype(str).str[:4][i+1]:
    historical_results_world_cup['is_final'][i] = 'TRUE'
  elif i < 899:
    historical_results_world_cup['is_final'][i] = 'FALSE'
  elif i == 899:
    historical_results_world_cup['is_final'][i] = 'TRUE'

/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2554105114.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_results_world_cup['is_final'][i] = 'TRUE'
/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2554105114.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_results_world_cup['is_final'][i] = 'FALSE'
/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2554105114.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [113]:
historical_results_world_cup.query('is_final == "TRUE"')

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,is_final
0,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False,TRUE
1,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False,TRUE
2,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False,TRUE
4,1879-04-07,Wales,Scotland,0.0,3.0,Friendly,Wrexham,Wales,False,TRUE
6,1880-03-27,Scotland,Wales,5.0,1.0,Friendly,Glasgow,Scotland,False,TRUE
...,...,...,...,...,...,...,...,...,...,...
782,1982-12-15,Yugoslavia,Wales,4.0,4.0,UEFA Euro qualification,Titograd,Yugoslavia,False,TRUE
793,1983-12-14,Wales,Yugoslavia,1.0,1.0,UEFA Euro qualification,Cardiff,Wales,False,TRUE
819,1984-12-10,South Korea,Qatar,0.0,1.0,AFC Asian Cup,Kallang,Singapore,True,TRUE
835,1985-10-16,Wales,Hungary,0.0,3.0,Friendly,Cardiff,Wales,False,TRUE


In [114]:
historical_results_world_cup['date'] = historical_results_world_cup['date'].astype('datetime64')

# Historical Results with possibily to win

Addying winner

In [115]:
historical_results_world_cup['winner'] = ''

for i in range(len(historical_results_world_cup['date'])):
  if historical_results_world_cup['home_score'][i] > historical_results_world_cup['away_score'][i]:
    historical_results_world_cup['winner'][i] = 'home_win'
  elif historical_results_world_cup['home_score'][i] < historical_results_world_cup['away_score'][i]:
    historical_results_world_cup['winner'][i] = 'away_win'
  else:
    historical_results_world_cup['winner'][i] = 'draw'

/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2780499933.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_results_world_cup['winner'][i] = 'home_win'
/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2780499933.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_results_world_cup['winner'][i] = 'away_win'
/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2780499933.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [116]:
historical_results_world_cup

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,is_final,winner
0,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win
1,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False,TRUE,away_win
2,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win
3,1879-01-18,England,Wales,2.0,1.0,Friendly,London,England,False,FALSE,home_win
4,1879-04-07,Wales,Scotland,0.0,3.0,Friendly,Wrexham,Wales,False,TRUE,away_win
...,...,...,...,...,...,...,...,...,...,...,...
2143,2022-06-14,Netherlands,Wales,3.0,2.0,UEFA Nations League,Rotterdam,Netherlands,False,,home_win
2144,2022-09-22,Belgium,Wales,2.0,1.0,UEFA Nations League,Brussels,Belgium,False,,home_win
2145,2022-09-23,Canada,Qatar,2.0,0.0,Friendly,Vienna,Austria,True,,home_win
2146,2022-09-25,Wales,Poland,0.0,1.0,UEFA Nations League,Cardiff,Wales,False,,away_win


In [117]:
historical_results_world_cup = historical_results_world_cup.merge(historical_win_loose_draw_ratios, left_on=['home_team', 'away_team'], right_on=['country1', 'country2'], how='left').rename(columns={
    'wins' : 'home_wins',
    'looses' : 'home_looses'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'tournament', 'city', 'country', 'neutral', 'is_final', 'winner', 'games', 'home_wins', 'home_looses', 'draws']]

In [118]:
historical_results_world_cup = historical_results_world_cup.replace(np.nan, 0) # adding for teams that never played before

In [119]:
historical_results_world_cup.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,is_final,winner,games,home_wins,home_looses,draws
0,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win,0.0,0.000000,0.000000,0.000000
1,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False,TRUE,away_win,0.0,0.000000,0.000000,0.000000
2,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win,0.0,0.000000,0.000000,0.000000
3,1879-01-18,England,Wales,2.0,1.0,Friendly,London,England,False,FALSE,home_win,104.0,0.653846,0.144231,0.201923
4,1879-04-07,Wales,Scotland,0.0,3.0,Friendly,Wrexham,Wales,False,TRUE,away_win,0.0,0.000000,0.000000,0.000000


# Historical result with ranking on date

Checking data

In [120]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             63916 non-null  int64  
 1   country_full     63916 non-null  object 
 2   country_abrv     63916 non-null  object 
 3   total_points     63916 non-null  float64
 4   previous_points  63916 non-null  float64
 5   rank_change      63916 non-null  int64  
 6   confederation    63916 non-null  object 
 7   rank_date        63916 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.9+ MB


In [121]:
ranking['country_full'].unique()

array(['Germany', 'Syria', 'Burkina Faso', 'Latvia', 'Burundi', 'Togo',
       'Angola', 'Suriname', 'Luxembourg',
       'St. Vincent and the Grenadines', 'Fiji', 'Mozambique',
       'Indonesia', 'Antigua and Barbuda', 'Jordan', 'Eswatini',
       'Faroe Islands', 'Lithuania', 'Uganda', 'Korea DPR', 'Peru',
       'Sierra Leone', 'Tanzania', 'Niger', 'Iraq', 'Guatemala',
       'Liberia', 'Ethiopia', 'Albania', 'Bolivia', 'Bahrain',
       'Singapore', 'Congo', 'Malaysia', 'Sudan', 'Croatia', 'Barbados',
       'Yemen', 'Cape Verde Islands', 'Solomon Islands', 'Libya',
       'Botswana', 'Brunei Darussalam', 'Dominican Republic', 'Lesotho',
       'India', 'Cuba', 'Pakistan', 'Belarus', 'Seychelles',
       'Central African Republic', 'Nepal', 'Gambia', 'Mauritania',
       'Kenya', 'Bangladesh', 'Slovenia', 'Oman', 'Guinea-Bissau',
       'St. Lucia', 'Hong Kong', 'Panama', 'Grenada', 'Puerto Rico',
       'Tahiti', 'South Africa', 'Venezuela', 'San Marino', 'Benin',
       'Guyana'

Adding home rank

In [122]:
historical_results_world_cup = historical_results_world_cup.merge(ranking, left_on=[historical_results_world_cup['date'].astype(str).str[:7], 'home_team'],
                                            right_on=[ranking['rank_date'].str[:7], 'country_full'],
                                            how='left').rename(columns={
                                                'rank' : 'home_rank',
                                                'total_points' : 'home_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'tournament', 'city', 'country', 'neutral', 'is_final', 'winner',
    'games', 'home_wins', 'home_looses', 'draws', 'home_rank', 'home_ranking_points']]

Adding away rank

In [123]:
historical_results_world_cup = historical_results_world_cup.merge(ranking, left_on=[historical_results_world_cup['date'].astype(str).str[:7], 'away_team'],
                                            right_on=[ranking['rank_date'].str[:7], 'country_full'],
                                            how='left').rename(columns={
                                                'rank' : 'away_rank',
                                                'total_points' : 'away_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'tournament', 'city', 'country', 'neutral', 'is_final', 'winner',
    'games', 'home_wins', 'home_looses', 'draws', 'home_rank', 'home_ranking_points',
    'away_rank', 'away_ranking_points']]

# Final historical dataset

In [124]:
historical_results_world_cup = historical_results_world_cup.replace(np.nan, 0) # replacing null ranks
historical_results_world_cup

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,is_final,winner,games,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points
0,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
1,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False,TRUE,away_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
2,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
3,1879-01-18,England,Wales,2.0,1.0,Friendly,London,England,False,FALSE,home_win,104.0,0.653846,0.144231,0.201923,0.0,0.00,0.0,0.00
4,1879-04-07,Wales,Scotland,0.0,3.0,Friendly,Wrexham,Wales,False,TRUE,away_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,2022-06-14,Netherlands,Wales,3.0,2.0,UEFA Nations League,Rotterdam,Netherlands,False,,home_win,10.0,1.000000,0.000000,0.000000,8.0,1679.41,19.0,1582.13
2150,2022-09-22,Belgium,Wales,2.0,1.0,UEFA Nations League,Brussels,Belgium,False,,home_win,16.0,0.375000,0.312500,0.312500,0.0,0.00,0.0,0.00
2151,2022-09-23,Canada,Qatar,2.0,0.0,Friendly,Vienna,Austria,True,,home_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
2152,2022-09-25,Wales,Poland,0.0,1.0,UEFA Nations League,Cardiff,Wales,False,,away_win,9.0,0.111111,0.666667,0.222222,0.0,0.00,0.0,0.00


In [125]:
historical_results_world_cup['home_rank'] = historical_results_world_cup['home_rank'].astype('float64')
historical_results_world_cup['home_ranking_points'] = historical_results_world_cup['home_ranking_points'].astype('float64')
historical_results_world_cup['away_rank'] = historical_results_world_cup['away_rank'].astype('float64')
historical_results_world_cup['away_ranking_points'] = historical_results_world_cup['away_ranking_points'].astype('float64')

In [126]:
historical_results_world_cup.corr()

/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/2415210706.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  historical_results_world_cup.corr()


,home_score,away_score,neutral,games,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points
home_score,1.000000,-0.103933,-0.066608,0.006478,0.118001,-0.139539,-0.053748,-0.077106,-0.042404,0.072494,-0.096447
away_score,-0.103933,1.000000,0.080148,0.056143,-0.107271,0.136212,-0.014289,-0.027649,-0.064469,-0.141943,-0.016468
neutral,-0.066608,0.080148,1.000000,-0.134447,0.047895,0.009726,0.075959,-0.279289,-0.064677,-0.267277,-0.054006
games,0.006478,0.056143,-0.134447,1.000000,0.420802,0.439408,0.440317,-0.143536,-0.078973,-0.146300,-0.078431
home_wins,0.118001,-0.107271,0.047895,0.420802,1.000000,0.266898,0.485733,-0.197804,0.061362,-0.104588,0.030581
home_looses,-0.139539,0.136212,0.009726,0.439408,0.266898,1.000000,0.479536,-0.064575,0.056663,-0.158203,0.095385
draws,-0.053748,-0.014289,0.075959,0.440317,0.485733,0.479536,1.000000,-0.123171,0.084160,-0.131267,0.075859
home_rank,-0.077106,-0.027649,-0.279289,-0.143536,-0.197804,-0.064575,-0.123171,1.000000,0.310910,0.718561,0.366234
home_ranking_points,-0.042404,-0.064469,-0.064677,-0.078973,0.061362,0.056663,0.084160,0.310910,1.000000,0.376947,0.896441
away_rank,0.072494,-0.141943,-0.267277,-0.146300,-0.104588,-0.158203,-0.131267,0.718561,0.376947,1.000000,0.294522


# Algoritmo

One hot encoding

In [127]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(historical_results_world_cup)
# Display the first 5 rows of the last 12 columns
features.iloc[:,5:].head(5)

,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points,home_team_Afghanistan,home_team_Albania,home_team_Algeria,...,country_Vietnam,country_Wales,country_Yemen,country_Yugoslavia,is_final_,is_final_FALSE,is_final_TRUE,winner_away_win,winner_draw,winner_home_win
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,0.653846,0.144231,0.201923,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0,0,...,0,1,0,0,0,0,1,1,0,0


Assigning label (what we want to predict) and converting to numpy

In [128]:
# Labels are the values we want to predict
train_labels = np.array(features.query('date < 2018')['home_score'])

# Remove the labels from the features
# axis 1 refers to the columns
train_features = features.query('date < 2018').drop('home_score', axis = 1).drop('date', axis = 1)
# Convert to numpy array
train_features = np.array(train_features)

In [129]:
test_labels = np.array(features.query('date >= 2018')['home_score'])
test_features = features.query('date >= 2018').drop('home_score', axis = 1).drop('date', axis = 1)
test_features = np.array(test_features)

Train and test split

In [130]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1973, 743)
Training Labels Shape: (1973,)
Testing Features Shape: (181, 743)
Testing Labels Shape: (181,)


Establishing a baseline error

In [131]:
# # The baseline predictions are the historical averages
# baseline_preds = test_features[:, feature_list.index('average')]
# # Baseline errors, and display average baseline error
# baseline_errors = abs(baseline_preds - test_labels)
# print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Predicting

In [132]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels);

In [133]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.58 degrees.


In [134]:
predictions.round(0)

array([1., 1., 2., 0., 0., 2., 0., 0., 3., 2., 1., 0., 2., 1., 0., 0., 2.,
       1., 1., 0., 1., 2., 2., 2., 0., 1., 3., 1., 0., 1., 2., 3., 0., 3.,
       2., 2., 1., 1., 2., 2., 1., 0., 0., 0., 1., 2., 0., 1., 1., 0., 0.,
       1., 0., 5., 2., 1., 1., 2., 3., 2., 1., 0., 1., 2., 0., 2., 2., 2.,
       3., 2., 2., 1., 2., 1., 5., 0., 2., 0., 0., 2., 2., 2., 1., 0., 1.,
       5., 1., 1., 2., 0., 1., 1., 3., 1., 2., 3., 2., 2., 2., 1., 5., 3.,
       2., 0., 1., 0., 1., 2., 4., 0., 2., 0., 0., 3., 3., 0., 0., 2., 3.,
       0., 2., 0., 0., 1., 2., 2., 2., 4., 2., 1., 2., 2., 1., 1., 2., 3.,
       0., 3., 1., 1., 1., 1., 2., 3., 1., 1., 4., 1., 0., 1., 1., 1., 1.,
       0., 2., 2., 0., 2., 2., 2., 2., 1., 2., 2., 2., 2., 0., 0., 2., 4.,
       0., 1., 3., 2., 0., 1., 3., 3., 1., 0., 2.])

In [135]:
df_predict_test = pd.merge(pd.DataFrame(predictions.round(0)), pd.DataFrame(test_labels), left_index=True, right_index=True).rename(columns={'0_x' : 'predicts', '0_y' : 'reality'})
df_predict_test['is_correct'] = df_predict_test['predicts'] - df_predict_test['reality']

df_predict_test.head(10)

,predicts,reality,is_correct
0,1.0,2.0,-1.0
1,1.0,0.0,1.0
2,2.0,2.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,2.0,5.0,-3.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,3.0,3.0,0.0
9,2.0,2.0,0.0


In [136]:
predict_right = ((df_predict_test['reality'].count() - df_predict_test.query('is_correct != 0')['is_correct'].count()) / df_predict_test['reality'].count() * 100).round(2)

print(f"The algorithm predicted right: {predict_right}% of the values")

The algorithm predicted right: 54.7% of the values


# 2022 calendar predictions

Reading and treating data

In [137]:
matches_schedule = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/matches-schedule.csv')

matches_schedule['date'] = matches_schedule['date'].str[-4:] + '-' + matches_schedule['date'].str[3:5] + '-' + matches_schedule['date'].str[:2]

matches_schedule['home_score'] = 0
matches_schedule['away_score'] = 0

matches_schedule.rename(columns={'country1' : 'home_team', 'country2' : 'away_team'}, inplace=True)

matches_schedule = matches_schedule[['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase']]

matches_schedule['date'] = matches_schedule['date'].astype('datetime64')

matches_schedule.head(5)

,date,home_team,home_score,away_score,away_team,phase
0,2022-11-21,Qatar,0,0,Ecuador,group matches
1,2022-11-21,Senegal,0,0,Netherlands,group matches
2,2022-11-21,England,0,0,Iran,group matches
3,2022-11-21,USA,0,0,Wales,group matches
4,2022-11-22,France,0,0,Australia,group matches


Adding historical ratio and rank

In [138]:
matches_schedule = matches_schedule.merge(historical_win_loose_draw_ratios, left_on=['home_team', 'away_team'], right_on=['country1', 'country2'], how='left').rename(columns={
    'wins' : 'home_wins',
    'looses' : 'home_looses'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'phase', 'games', 'home_wins', 'home_looses', 'draws']]

In [139]:
matches_schedule

,date,home_team,away_team,home_score,away_score,phase,games,home_wins,home_looses,draws
0,2022-11-21,Qatar,Ecuador,0,0,group matches,3.0,0.333333,0.333333,0.333333
1,2022-11-21,Senegal,Netherlands,0,0,group matches,NaN,NaN,NaN,NaN
2,2022-11-21,England,Iran,0,0,group matches,NaN,NaN,NaN,NaN
3,2022-11-21,USA,Wales,0,0,group matches,NaN,NaN,NaN,NaN
4,2022-11-22,France,Australia,0,0,group matches,5.0,0.600000,0.200000,0.200000
5,2022-11-22,Denmark,Tunisia,0,0,group matches,1.0,1.000000,0.000000,0.000000
6,2022-11-22,Mexico,Poland,0,0,group matches,9.0,0.333333,0.333333,0.333333
7,2022-11-22,Argentina,Saudi Arabia,0,0,group matches,4.0,0.500000,0.000000,0.500000
8,2022-11-23,Belgium,Canada,0,0,group matches,1.0,1.000000,0.000000,0.000000
9,2022-11-23,Spain,Costa Rica,0,0,group matches,3.0,0.666667,0.000000,0.333333


In [140]:
matches_schedule = matches_schedule.merge(ranking.query('rank_date.astype("datetime64") > 2021').groupby('country_full').mean().round(0),
                                          left_on='home_team',
                                          right_on='country_full',
                                          how='left').rename(columns={
                                                                      'rank' : 'home_rank',
                                                                      'total_points' : 'home_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'phase', 'games', 'home_wins', 'home_looses', 'draws', 'home_rank', 'home_ranking_points']]

/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/4132625271.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  matches_schedule = matches_schedule.merge(ranking.query('rank_date.astype("datetime64") > 2021').groupby('country_full').mean().round(0),


In [141]:
matches_schedule = matches_schedule.merge(ranking.query('rank_date.astype("datetime64") > 2021').groupby('country_full').mean().round(0),
                                          left_on='away_team',
                                          right_on='country_full',
                                          how='left').rename(columns={
                                                                      'rank' : 'away_rank',
                                                                      'total_points' : 'away_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'phase', 'games', 'home_wins', 'home_looses', 'draws', 'home_rank',
    'home_ranking_points', 'away_rank', 'away_ranking_points']]

/var/folders/wx/h8jjlc4161d05l98dzwm054w0000gn/T/ipykernel_14555/1233119693.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  matches_schedule = matches_schedule.merge(ranking.query('rank_date.astype("datetime64") > 2021').groupby('country_full').mean().round(0),


In [142]:
matches_schedule = matches_schedule.fillna(0)

Final data for schedule

In [143]:
matches_schedule.head(10)

,date,home_team,away_team,home_score,away_score,phase,games,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points
0,2022-11-21,Qatar,Ecuador,0,0,group matches,3.0,0.333333,0.333333,0.333333,50.0,1431.0,49.0,1436.0
1,2022-11-21,Senegal,Netherlands,0,0,group matches,0.0,0.000000,0.000000,0.000000,20.0,1567.0,11.0,1647.0
2,2022-11-21,England,Iran,0,0,group matches,0.0,0.000000,0.000000,0.000000,4.0,1733.0,0.0,0.0
3,2022-11-21,USA,Wales,0,0,group matches,0.0,0.000000,0.000000,0.000000,15.0,1620.0,19.0,1574.0
4,2022-11-22,France,Australia,0,0,group matches,5.0,0.600000,0.200000,0.200000,3.0,1769.0,38.0,1478.0
5,2022-11-22,Denmark,Tunisia,0,0,group matches,1.0,1.000000,0.000000,0.000000,10.0,1651.0,29.0,1510.0
6,2022-11-22,Mexico,Poland,0,0,group matches,9.0,0.333333,0.333333,0.333333,11.0,1647.0,25.0,1540.0
7,2022-11-22,Argentina,Saudi Arabia,0,0,group matches,4.0,0.500000,0.000000,0.500000,5.0,1727.0,55.0,1413.0
8,2022-11-23,Belgium,Canada,0,0,group matches,1.0,1.000000,0.000000,0.000000,1.0,1816.0,50.0,1430.0
9,2022-11-23,Spain,Costa Rica,0,0,group matches,3.0,0.666667,0.000000,0.333333,7.0,1689.0,43.0,1457.0


## Running algorithm

In [144]:
historical_results_world_cup

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,is_final,winner,games,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points
0,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
1,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False,TRUE,away_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
2,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False,TRUE,home_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
3,1879-01-18,England,Wales,2.0,1.0,Friendly,London,England,False,FALSE,home_win,104.0,0.653846,0.144231,0.201923,0.0,0.00,0.0,0.00
4,1879-04-07,Wales,Scotland,0.0,3.0,Friendly,Wrexham,Wales,False,TRUE,away_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,2022-06-14,Netherlands,Wales,3.0,2.0,UEFA Nations League,Rotterdam,Netherlands,False,,home_win,10.0,1.000000,0.000000,0.000000,8.0,1679.41,19.0,1582.13
2150,2022-09-22,Belgium,Wales,2.0,1.0,UEFA Nations League,Brussels,Belgium,False,,home_win,16.0,0.375000,0.312500,0.312500,0.0,0.00,0.0,0.00
2151,2022-09-23,Canada,Qatar,2.0,0.0,Friendly,Vienna,Austria,True,,home_win,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00
2152,2022-09-25,Wales,Poland,0.0,1.0,UEFA Nations League,Cardiff,Wales,False,,away_win,9.0,0.111111,0.666667,0.222222,0.0,0.00,0.0,0.00


In [145]:
teams_to_query = matches_schedule['home_team']

# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(
    historical_results_world_cup.query('home_team in @teams_to_query').query(
        'away_team in @teams_to_query'
    )[['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'games', 'home_wins', 'home_looses', 'draws', 'home_rank',
       'home_ranking_points', 'away_rank', 'away_ranking_points']]
    )
    
features_to_predict = pd.get_dummies(
    matches_schedule[['date', 'home_team', 'away_team', 'home_score', 'away_score',
                      'games', 'home_wins', 'home_looses', 'draws', 'home_rank',
                      'home_ranking_points', 'away_rank', 'away_ranking_points']])

# Labels are the values we want to predict
train_labels = np.array(features['home_score'])

# Remove the labels from the features
# axis 1 refers to the columns
train_features = features.drop('home_score', axis = 1).drop('date', axis = 1)
# Convert to numpy array
train_features = np.array(train_features)

test_labels = np.array(features_to_predict['home_score'])
test_features = features_to_predict.drop('home_score', axis = 1).drop('date', axis = 1)
test_features = np.array(test_features)

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels)

# print('Training Features Shape:', train_features.shape)
# print('Training Labels Shape:', train_labels.shape)
# print('Testing Features Shape:', test_features.shape)
# print('Testing Labels Shape:', test_labels.shape)

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

In [150]:
matches_schedule_predicted = pd.merge(pd.DataFrame(predictions.round(0)), matches_schedule, left_index=True, right_index=True)

matches_schedule_predicted.rename(columns={0 : "home_score_predicted"}, inplace=True)

matches_schedule_predicted = matches_schedule_predicted[['date', 'home_team', 'home_score_predicted', 'away_score', 'away_team']]

matches_schedule_predicted

,date,home_team,home_score_predicted,away_score,away_team
0,2022-11-21,Qatar,1.0,0,Ecuador
1,2022-11-21,Senegal,1.0,0,Netherlands
2,2022-11-21,England,1.0,0,Iran
3,2022-11-21,USA,1.0,0,Wales
4,2022-11-22,France,2.0,0,Australia
5,2022-11-22,Denmark,4.0,0,Tunisia
6,2022-11-22,Mexico,1.0,0,Poland
7,2022-11-22,Argentina,1.0,0,Saudi Arabia
8,2022-11-23,Belgium,2.0,0,Canada
9,2022-11-23,Spain,2.0,0,Costa Rica
